In [1]:
!pip install selenium

In [2]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv

In [34]:
from selenium import webdriver

driver = webdriver.Chrome()
driver.set_window_size(1200, 800)
# driver.get('https://www.scopus.com/')
driver.get('https://id.elsevier.com/as/authorization.oauth2?platSite=SC%2Fscopus&ui_locales=en-US&scope=openid+profile+email+els_auth_info+els_analytics_info+urn%3Acom%3Aelsevier%3Aidp%3Apolicy%3Aproduct%3Aindv_identity&els_policy=idp_policy_indv_identity_plus&response_type=code&redirect_uri=https%3A%2F%2Fwww.scopus.com%2Fauthredirect.uri%3FtxGid%3Dfb217ed23f1a8fa89a3c25353265969c&state=userLogin%7CtxId%3DBDE7586B5426BBB2865B7D800E2A8989.i-010cd1cc0cb8b14af%3A5&authType=SINGLE_SIGN_IN&prompt=login&client_id=SCOPUS')

In [6]:
def search_doi(doi):
    search_with = driver.find_element(By.CSS_SELECTOR, 'select[data-testid="select-field-select"]')
    search_with.send_keys("DOI")

    search_box = driver.find_element(By.CSS_SELECTOR, 'input[data-testid="search-field-input"]')
    search_box.clear()
    search_box.send_keys(doi)

    search_button = driver.find_element(By.CSS_SELECTOR, 'button[class="Button-module__nc6_8 Button-module__rphhF Button-module__VBKvn Button-module__R359q Button-module__hK_LA Button-module__x5a4w Button-module__rTQlw"]')
    search_button.click()

    divSearch = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'table > tbody > tr > td > div > div > h3')))
    # tbody = WebDriverWait(divSearch, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "tbody")))
    # h3 = WebDriverWait(tbody, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "h3")))

    # divSearch = driver.find_element(By.CSS_SELECTOR, 'div[class="document-results-list-layout"]')
    # tbody = divSearch.find_element(By.CSS_SELECTOR, "tbody")
    # h3 = tbody.find_element(By.CSS_SELECTOR, "h3")
    divSearch.click()

In [ ]:
def get_element(csv_info, csv_relation, csv_level, run_num):
    doi_ele = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'dl[data-testid="source-info-entry-doi"] > dd')))
    # doi_ele = driver.find_element(By.CSS_SELECTOR, 'dl[data-testid="source-info-entry-doi"] > dd')
    doi = doi_ele.text
    print('DOI >>',doi)


    title_ele = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'h2[data-testid="publication-titles"]')))
    title = title_ele.text
    print("Title >>",title)

    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'li[class="DocumentHeader-module__Ltfqf"]')))
    subDeEles = driver.find_elements(By.CSS_SELECTOR, 'li[class="DocumentHeader-module__Ltfqf"] > span')
    
    province = []
    country = []
    for lo in subDeEles:
        lo_t = lo.text
        split_location = lo_t.split(',')
        if len(split_location) >= 3:
            sub_country = split_location[-1].strip()
            country.append(sub_country)
            sub_province = split_location[-3].strip() if split_location[-2].strip().isdigit() else split_location[-2].strip()
            province.append(sub_province)
    print("Country >>", country)
    print("Province >>", province)


    cited = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'h3[class="panel-title"]')))
    citedCount = cited.text.split()[2]
    print('Cited count >>',citedCount)

    refTable = []
    try:
        lenRef = WebDriverWait(driver, 2).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'ul[class="refTitleCount list-inline"] > li > h4')))
        print("Ref show >>",lenRef.text.split("(")[-1].strip(")"))
    
        tbodyTable = WebDriverWait(driver, 1).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'tbody[class="referencesUL"]')))
        refTable = tbodyTable.find_elements(By.CSS_SELECTOR, 'td')
    except Exception as e:
        print('boom')

    ref_doi = []
    ref_doi_link = []
    ref_text = []
    print("Ref count >>",len(refTable))

    for ref in refTable:
        if "doi:" in ref.text:
            doiX = ref.text.split("doi:")[-1].strip()
            doiX_sp = doiX.split()
            sub_ref_doi = ''
            if len(doiX_sp) >= 1:
                sub_ref_doi = doiX.split()[0]
            # print("ref d :",doi) 
            try:
                divLink = ref.find_element(By.CSS_SELECTOR, 'div[class="refDocTitle fontMedium"] > a')
                link = divLink.get_attribute('href')
                # print(link)
                ref_doi_link.append(link)
            except Exception as e:
                text = ref.text.splitlines()[1]
                # print("ref t :",text)
                ref_text.append(text)
                ref_doi_link.append(None)
            ref_doi.append(sub_ref_doi)
        else:
            text = ref.text.splitlines()[1]
            # print("ref t :",text)
            ref_text.append(text)

    print("Ref DOI >>",ref_doi)
    print("Ref DOI link >>",ref_doi_link)
    print("Ref Text >>",ref_text)

    with open(csv_info, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        row = [run_num, True, doi, title, country, province, citedCount]
        print("info =>",row)
        writer.writerow(row)

    with open(csv_relation, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        row = [run_num, True, doi, None, ref_doi, ref_text, len(ref_doi), len(ref_text)]
        print("relation =>",row)
        writer.writerow(row)

    with open(csv_level, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)

        for j in range(0,len(ref_doi_link)):
            row = [run_num, doi, ref_doi[j], ref_doi_link[j], 0]
            writer.writerow(row)
            print("~",end='')
        print()

In [8]:
def one_flow(doi, csv_info, csv_relation, csv_level, run_num):
    driver.get('https://www.scopus.com/')
    search_doi(doi)
    get_element(csv_info, csv_relation, csv_level, run_num)

In [48]:
import pandas as pd

df = pd.read_csv("jan_csv_0.csv")

# df["prism:doi"].isna().sum()

df = df.dropna(subset=["prism:doi"]).reset_index()

for i in range(536,1000):
    print(f"""----------{i}----------""")
    try:
        doi = df["prism:doi"][i]
        one_flow(doi,'aa_info.csv','aa_relation.csv','aa_level_0.csv', i)
    except Exception as e:
        print(e)
        with open("aa_cash.csv", mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            row = [df["prism:doi"][i]]
            writer.writerow(row)

----------536----------
DOI >> 10.1007/s12640-018-9906-2
Title >> Generalized Anxiety Disorder (GAD) and Comorbid Major Depression with GAD Are Characterized by Enhanced Nitro-oxidative Stress, Increased Lipid Peroxidation, and Lowered Lipid-Associated Antioxidant Defenses
Country >> ['Brazil', 'Thailand', 'Bulgaria', 'Australia']
Province >> ['Parana', 'Bangkok', 'Plovdiv', 'VIC']
Cited count >> 46
Ref show >> 136
Ref count >> 80
Ref DOI >> ['10.1016/S0076-6879(85)13073-9', '10.1016/j.jad.2008.04.013', '10.4103/0253-7176.85403', '10.1016/j.abb.2010.04.008', '10.1007/s00406-004-0484-3', '10.1016/j.jpsychires.2016.07.007', '10.1016/j.ab.2012.01.016', '10.1016/S0165-0327(00)00199-3', '10.1016/j.fertnstert.2011.04.016', '10.1016/j.jad.2017.09.003', '10.1016/j.jad.2014.02.018', '10.1179/1351000214Y.0000000093', '10.1016/j.psychres.2014.04.005', '10.1016/j.jad.2013.03.011', '10.1016/j.biopsych.2009.02.020', '10.3109/13651501.2015.1016973', '10.1038/nrdp.2017.24', '10.1017/s1092852900002923'

In [44]:
doi = '10.1016/j.compchemeng.2018.07.019'
one_flow(doi,'aa_info.csv','aa_relation.csv','aa_level_0.csv', 300)

DOI >> 10.1016/j.compchemeng.2018.07.019
Title >> Integration of the biorefinery concept for the development of sustainable processes for pulp and paper industry
Country >> ['Thailand', 'Denmark', 'Thailand']
Province >> ['Bangkok', 'DK', 'Bangkok']
Cited count >> 54
Ref show >> 71
Ref count >> 71
Ref DOI >> ['10.1016/j.cherd.2009.07.006', '10.1016/j.egypro.2014.01.218', '10.1016/j.compchemeng.2015.04.030', '10.1016/j.biombioe.2013.08.022', '10.1016/j.compchemeng.2017.01.030', '10.1039/b922014c', '10.1016/j.ymben.2014.12.007', '10.1016/j.cherd.2009.07.017', '10.1002/bbb.105', '10.1016/j.energy.2003.09.005', '10.1080/01614940903048513', '10.1016/B978-0-444-63472-6.00007-0', '10.1016/j.seppur.2009.05.001', '10.1099/00207713-49-1-207', '10.1002/bbb.1468', '10.1016/j.biortech.2009.07.092', '10.1016/S0960-8524(96)00087-9', '10.1016/j.energy.2016.02.091', '', '10.1016/j.compchemeng.2016.01.006', '10.1007/s10529-009-0163-6', '10.1023/A:1021907116361', '10.1007/s00253-002-0935-6', '10.1016/j.i